In [5]:
import pandas as pd
data = pd.read_csv("~/Desktop/flashcard_/python files/Students_quizzes.csv")
data

,group,instructor,quiz_average,Exam_1,Total,Grade
0,1,B,5.0,76,101.0,71.6710
1,1,B,5.0,77,102.0,72.5045
2,1,B,2.5,0,12.5,4.1625
3,1,B,5.7,92,120.5,86.1725
4,1,B,9.5,87,134.5,88.3320
...,...,...,...,...,...,...
288,13,C,4.6,33,56.0,35.1645
289,13,C,6.0,54,84.0,54.9990
290,13,C,3.0,33,48.0,32.5005
291,13,C,9.0,96,141.0,95.0010


# EDA

In [9]:
data.describe()
data["group"].value_counts()
data["instructor"].value_counts()
data[["instructor","quiz_average"]].groupby("instructor").mean()

,quiz_average
instructor,
A,3.881159
B,4.989063
C,8.086667
D,5.612857
